# Homework 3 - MapReduce

In this homework, we are practicing the MapReduce programming paradigm. 

You are required to turn in this notebook as BDM\_HW3\_MR\_**NetId**.ipynb. You will be asked to complete each task using the accompanied *mapreduce* package (as **mapreduce.py**) and/or MRJob using one or more MapReduce "steps". For each such step (with **mr.run()**), you are expected to supply a mapper and a reducer as needed. Or if you're using MRJob, please call **mr.runJob()** instead. Below are sample usage of the package:

```python
    # Run on input1 using your mapper1 and reducer1 function
    output = list(mr.run(input1, mapper1, reducer1))

    # Run on input2 using only your mapper2, no reduce phase
    output = list(mr.run(enumerate(input2), mapper2, combiner2))
    
    # Run on input3 using 2 nested MapReduce jobs
    output = mr.run(mr.run(input3, mapper3, reducer3), mapper4)
```
    
Please note that the input must be an iteratable of **key/value pairs**. If your inpu tdata does not have a key, you can simply add a null or index key through **enumerator(input)**. The output of the mr.run() is always a **generator**. You have to cast it to a list if you'd like to view, index or print it out.

## Task (10 points)

There is only one task in this homework. You are asked to implement the Social Triangle example discussed in class. In particular, given the email dataset, please list all "reciprocal" relationships in the company. Recall that:

If A emails B and B emails A, then A and B is *reciprocal*.

If A emails B but B doesn’t email A, then A and B is *directed*.

**Dataset:** We will use a subset of the open [Enron Email Dataset](https://www.cs.cmu.edu/~./enron/ "Enron Email Dataset"), which contains approximately 10,000 simplified email headers from the Enron Corporation. You can download this dataset from NYU Classes as **enron_mails_small.csv**. The file contains 3 columns *Date*, *From*, and *To*. Their description is as follows:

|Column name|Description|
|--|--|
|Date |The date and time of the email, in the format YYYY-MM-DD hh-mm-ss, <br />e.g. "1998-10-30 07:43:00" |
|From |The sender email address, <br />e.g. "mark.taylor@enron.com" |
|To | A list of recipients' email addresses separated by semicolons ';', <br />e.g. "jennifer.fraser@enron.com;jeffrey.hodge@enron.com" |

Note that, we only care about users employed by Enron, or only relationships having email addresses that end with *'@enron.com'*.

The expected output is also provided below. For each reciprocal relationship, please output a tuple consisting of two strings. The first one is always **'reciprocal'**. And the second one is a string showing the name of the two person in the following format: **'Jane Doe : John Doe'**. The names should be presented in the lexical order, i.e. there will not be a 'John Doe : Jane Doe' since 'Jane' is ordered before 'John.

Though the dataset only contains email addresses, not actual names, we're assuming that the email aliases were created based on their name. For example:

|Email Address|Converted Name|
|--|--|
|mark.taylor@enron.com|Mark Taylor|
|alan.aronowitz@enron.com|Alan Aronowitz|
|marc.r.cutler@enron.com|Marc R Cutler|
|hugh@enron.com|Hugh|

Please fill the code block with a series of MapReduce jobs using your own mapper and reducer functions. Be sure to include the naming convention logic into one of your mappers and/or reducers.

In [1]:
import csv
import mapreduce as mr

def mapper1(_,row):
    #map a sender to a list of receivers from each email, sender is the key, the list of receivers is the value
    if row['From'].endswith('@enron.com'):
        for i in row['To'].split(";"):
            if i.endswith('@enron.com'):
                yield (row['From'],i)
    
def reducer1(sender,receivers):
    # group by senders, put all the receviers from all emails into a set to remove dupicates
    yield (sender,set(receivers)) 
    

def mapper2(_,row):
    # map each receiver to a sender from every email, each receiver is a key and the send is the value 
    if row['From'].endswith('@enron.com'):
        for i in row['To'].split(";"):
            if i.endswith('@enron.com'):
                yield (i,row['From'])
        
def reducer2(receiver,senders):
    # group by receivers, put all the senders from all emails into a set to remove dupicates
    yield ((receiver,set(senders)))   


def mapper3(_,row):
    # joining the two lists
    yield (row[0],row[1])

def reducer3(person1,combined_lists):
    # group by each person, combine the receivers list and the senders list
    yield (person1,combined_lists)


def mapper4(person1,combined_lists):
    # find person with the recipropcals relationship in their combined receiver_sender_lists
    if len(combined_lists)==2 and len(combined_lists[0]&combined_lists[1])!=0:
        recipropcals = combined_lists[0]&combined_lists[1]
        yield (person1,recipropcals)

def reducer4(person1,recipropcals):
    #remove the person himself/herself from the recipropcals list, sort the sender and receiver in the lexical order
    for i in recipropcals:
        for person2 in i:
            if person2!= person1:
                yield(tuple(sorted((person1,person2))))
    
def mapper5(person1,person2):
    # Transfer email addresses to actual names and finalize the output 
    if len(person1.split('@')[0].split('.')) == 2:
        name1 = person1.split('@')[0].split('.')[0].capitalize()+" "+person1.split('@')[0].split('.')[1].capitalize()
    if len(person2.split('@')[0].split('.')) == 2:
        name2 = person2.split('@')[0].split('.')[0].capitalize()+" "+person2.split('@')[0].split('.')[1].capitalize()
    else:
        name1 = person1.split('@')[0].replace("_", " ").capitalize()
        name2 = person2.split('@')[0].replace("_", " ").capitalize()
        
    pair = name1+" : "+name2
    
    yield(("recipropcal",pair))
    
    
with open('enron_mails_small.csv', 'r') as fi:
    reader = enumerate(csv.DictReader(fi))
    output1=  list(mr.run(reader, mapper1,reducer1))
    
with open('enron_mails_small.csv', 'r') as fi:
    reader = enumerate(csv.DictReader(fi))
    output2 = list(mr.run(reader, mapper2,reducer2))

reader = enumerate(output1+output2)
output = sorted(list(mr.run(set(sorted(mr.run(mr.run(reader, mapper3,reducer3),mapper4,reducer4))),mapper5)))
print(len(output))
output

35


[('recipropcal', 'Brenda Whitehead : Elizabeth Sager'),
 ('recipropcal', 'Carol Clair : Debra Perlingiere'),
 ('recipropcal', 'Carol Clair : Mark Taylor'),
 ('recipropcal', 'Carol Clair : Richard Sanders'),
 ('recipropcal', 'Carol Clair : Sara Shackleton'),
 ('recipropcal', 'Carol Clair : Tana Jones'),
 ('recipropcal', 'Debra Perlingiere : Kevin Ruscitti'),
 ('recipropcal', 'Drew Fossum : Susan Scott'),
 ('recipropcal', 'Elizabeth Sager : Janette Elbertson'),
 ('recipropcal', 'Elizabeth Sager : Mark Haedicke'),
 ('recipropcal', 'Elizabeth Sager : Mark Taylor'),
 ('recipropcal', 'Elizabeth Sager : Richard Sanders'),
 ('recipropcal', 'Eric Bass : Susan Scott'),
 ('recipropcal', 'Fletcher Sturm : Greg Whalley'),
 ('recipropcal', 'Fletcher Sturm : Sally Beck'),
 ('recipropcal', 'Gerald Nemec : Susan Scott'),
 ('recipropcal', 'Grant Masson : Vince Kaminski'),
 ('recipropcal', 'Greg Whalley : Richard Sanders'),
 ('recipropcal', 'Janette Elbertson : Mark Taylor'),
 ('recipropcal', 'Janette El